##Rozwiązanie problemu n hetmanów za pomosą SAT-solvera

### Setup

In [1]:
!pip install pycosat
!pip install python-sat[pblib,aiger]

In [2]:
import pycosat
from pysat.solvers import Solver

_default_solver = 'mgh'  #seems fastests for n under 12
allow_three_axes = False

#General utility funcitons

def redu(x,y,_n):
  return x + _n*(y-1)

def redu3(x,y,z,_n):
  return x + _n*(y-1) + _n*_n*(z-1)

def inGrid(x,_n):
  return 0<x and x<=_n

def plotsol(x,_n):
  if(type(x) == str):
    print(x)
  else:
    for v in x:
      if v > 0:
        print("X",end=' ')
      else:
        print(".",end=' ')
      if(v%_n == 0):
        print("\n")

#Generating CNF form of our problem

def get_cnf(_n, dim=2):
  if dim == 2:
    cnf = []
    cnf += [[redu(x,y,_n) for x in range(1,_n+1)] for y in range(1,_n+1)]
    cnf += [[-redu(i,j,_n),-redu(i,k,_n)] for i in range(1,_n+1) for j in range(1,_n+1) for k in range(1,_n+1) if (k!=j and k<j)]
    cnf += [[-redu(i,j,_n),-redu(k,j,_n)] for i in range(1,_n+1) for j in range(1,_n+1) for k in range(1,_n+1) if (k!=i and k<i)]
    cnf += [[-redu(k+i-1,i,_n),-redu(k+j-1,j,_n)] for k in range(-_n,_n+1) for i in range(1,_n+1) for j in range(1,_n+1) if (i!=j and i<j and inGrid(k+i-1,_n) and inGrid(k+j-1,_n))]
    cnf += [[-redu(k-i+1,i,_n),-redu(k-j+1,j,_n)] for k in range(0,_n+_n+2) for i in range(1,_n+1) for j in range(1,_n+1) if (i!=j and i<j and inGrid(k-i+1,_n) and inGrid(k-j+1,_n))]
    return cnf
  elif dim == 3: #n^2 queens on n^3 spaces
    cnf = []
    cnf += [[redu3(x,y,z,_n) for x in range(1,_n+1)] for y in range(1,_n+1) for z in range(1,_n+1)]
    #One axis
    cnf += [[-redu3(i,j,z,_n),-redu3(i,k,z,_n)] for i in range(1,_n+1) for j in range(1,_n+1) for k in range(1,_n+1) for z in range(1,_n+1) if (k!=j and k<j)]
    cnf += [[-redu3(i,j,z,_n),-redu3(k,j,z,_n)] for i in range(1,_n+1) for j in range(1,_n+1) for k in range(1,_n+1) for z in range(1,_n+1) if (k!=i and k<i)]
    cnf += [[-redu3(i,j,z,_n),-redu3(i,j,k,_n)] for i in range(1,_n+1) for j in range(1,_n+1) for k in range(1,_n+1) for z in range(1,_n+1) if (k!=z and k<z)]
    #Two axes
    cnf += [[-redu3(k+i-1,i,z,_n),-redu3(k+j-1,j,z,_n)] for k in range(-_n, _n+1) for i in range(1,_n+1) for j in range(1,_n+1) for z in range(1,_n+1) if (i!=j and i<j and inGrid(k+i-1,_n) and inGrid(k+j-1,_n))]
    cnf += [[-redu3(k-i+1,i,z,_n),-redu3(k-j+1,j,z,_n)] for k in range(0,_n+_n+2) for i in range(1,_n+1) for j in range(1,_n+1) for z in range(1,_n+1) if (i!=j and i<j and inGrid(k-i+1,_n) and inGrid(k-j+1,_n))]
    cnf += [[-redu3(k+i-1,z,i,_n),-redu3(k+j-1,z,j,_n)] for k in range(-_n, _n+1) for i in range(1,_n+1) for j in range(1,_n+1) for z in range(1,_n+1) if (i!=j and i<j and inGrid(k+i-1,_n) and inGrid(k+j-1,_n))]
    cnf += [[-redu3(k-i+1,z,i,_n),-redu3(k-j+1,z,j,_n)] for k in range(0,_n+_n+2) for i in range(1,_n+1) for j in range(1,_n+1) for z in range(1,_n+1) if (i!=j and i<j and inGrid(k-i+1,_n) and inGrid(k-j+1,_n))]
    cnf += [[-redu3(z,k+i-1,i,_n),-redu3(z,k+j-1,j,_n)] for k in range(-_n, _n+1) for i in range(1,_n+1) for j in range(1,_n+1) for z in range(1,_n+1) if (i!=j and i<j and inGrid(k+i-1,_n) and inGrid(k+j-1,_n))]
    cnf += [[-redu3(z,k-i+1,i,_n),-redu3(z,k-j+1,j,_n)] for k in range(0,_n+_n+2) for i in range(1,_n+1) for j in range(1,_n+1) for z in range(1,_n+1) if (i!=j and i<j and inGrid(k-i+1,_n) and inGrid(k-j+1,_n))]
    #Three axes
    if(allow_three_axes): 
      cnf += [[-redu3(i,j,z,_n),-redu3(i+m,j+m,z+m,_n)] for i in range(1,_n+1) for j in range(1,_n+1) for z in range(1,_n+1) for m in range(1,_n+1) if (m!=0 and inGrid(i+m,_n) and inGrid(j+m,_n) and inGrid(z+m,_n))]
      cnf += [[-redu3(i,j,z,_n),-redu3(i-m,j+m,z+m,_n)] for i in range(1,_n+1) for j in range(1,_n+1) for z in range(1,_n+1) for m in range(1,_n+1) if (m!=0 and inGrid(i-m,_n) and inGrid(j+m,_n) and inGrid(z+m,_n))]
      cnf += [[-redu3(i,j,z,_n),-redu3(i+m,j-m,z+m,_n)] for i in range(1,_n+1) for j in range(1,_n+1) for z in range(1,_n+1) for m in range(1,_n+1) if (m!=0 and inGrid(i+m,_n) and inGrid(j-m,_n) and inGrid(z+m,_n))]
      cnf += [[-redu3(i,j,z,_n),-redu3(i+m,j+m,z-m,_n)] for i in range(1,_n+1) for j in range(1,_n+1) for z in range(1,_n+1) for m in range(1,_n+1) if (m!=0 and inGrid(i+m,_n) and inGrid(j+m,_n) and inGrid(z-m,_n))]
    return cnf
  else:
    return []


#Pycosat functions

def solve_pycosat(_n,_name='no'):
  plotsol(pycosat.solve(get_cnf(_n)),_n)

def num_solutions_pycosat(_n,_name='no'):
  return len(list(pycosat.itersolve(get_cnf(_n))))

def solvable_pycosat(_n,_name='no'):
  _x = pycosat.solve(get_cnf(_n))
  if(type(_x) == str):
    return False
  else:
    return True

#Pysat functions

def solve_pysat(_n , _name=_default_solver):
  s1 = Solver(name=_name, bootstrap_with=get_cnf(_n))
  if s1.solve() == True:
    plotsol(s1.get_model(),_n)
  else:
    print("UNSOL")
  s1.delete()

def num_solutions_pysat(_n, _name=_default_solver):
  s1 = Solver(name=_name, bootstrap_with=get_cnf(_n))
  s1.solve()
  _x = len(list(s1.enum_models()))
  s1.delete()
  return _x

def solvable_pysat(_n,_name=_default_solver):
  s1 = Solver(name=_name, bootstrap_with=get_cnf(_n))
  if s1.solve() == True:
    s1.delete()
    return True
  else:
    s1.delete()
    return False

###Code execution

In [3]:
import time

n = 10

start_time = time.time()
_s = num_solutions_pycosat(n)
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
_s = num_solutions_pysat(n,'m22')
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
_s = num_solutions_pysat(n,'g3')
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
_s = num_solutions_pysat(n,'mgh')
print("--- %s seconds ---" % (time.time() - start_time))

del(_s)

--- 0.5235493183135986 seconds ---
--- 0.31572437286376953 seconds ---
--- 0.5808887481689453 seconds ---
--- 0.3053097724914551 seconds ---


In [4]:
LIM = 25
print('Solvability for n up to ' + str(LIM))
for i in range(1,LIM+1):
  print(i,' ' if i<10 else '',solvable_pysat(i))

Solvability for n up to 25
1   True
2   False
3   False
4   True
5   True
6   True
7   True
8   True
9   True
10  True
11  True
12  True
13  True
14  True
15  True
16  True
17  True
18  True
19  True
20  True
21  True
22  True
23  True
24  True
25  True


In [5]:
LIM = 12
print('Number of solutions for n up to ' + str(LIM))
for i in range(1,LIM+1):
  print(i,num_solutions_pysat(i))

Number of solutions for n up to 12
1 1
2 0
3 0
4 2
5 10
6 4
7 40
8 92
9 352
10 724
11 2680
12 14200


In [6]:
LIM = 11
print('3D Solvability for n up to ' + str(LIM))
for i in range(1,LIM+1):
  m = i
  s1 = Solver(name=_default_solver, bootstrap_with=get_cnf(m,dim=3))
  print(i,' ' if i<10 else '',s1.solve())
  s1.delete()

3D Solvability for n up to 11
1   True
2   False
3   False
4   False
5   False
6   False
7   True
8   False
9   False
10  False
11  True


In [ ]:
LIM = 11
print('3D Number of solutions for n up to ' + str(LIM))
for i in range(1,LIM+1):
  m = i
  s1 = Solver(name=_default_solver, bootstrap_with=get_cnf(m,dim=3))
  s1.solve();
  print(i,' ' if i<10 else '',len(list(s1.enum_models())))
  s1.delete()

3D Number of solutions for n up to 11
1   1
2   0
3   0
4   0
5   0
6   0
7   56
8   0
9   0
10  0
11  528


In [ ]:
get_cnf(4)